## GroupBy  
O nome "group by" vem de um comando na linguagem do banco de dados SQL, mas talvez seja mais esclarecedor pensar nisso nos termos cunhados primeiramente por Hadley Wickham, da Rstats: **split, apply, combine** (dividir, aplicar, combinar).

![](Split, apply, combine.png)

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns # Vamos importar o seaborn apenas para utilizar o dataset planet

In [2]:
planets = sns.load_dataset('planets')
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


**GroupBy**  
Podemos calcular a operação mais básica split-apply-combine com o método ``groupby()`` de DataFrames, passando o nome da coluna-chave desejada.  
A função ``groupby()`` retornará somente um objeto (DataFrameGroupBy). Para visualizarmos o resultado necessitamos informar o agregador para esse objeto, que pode ser ``sum()``, ``median()``, ``count()``, etc. 

In [3]:
# No exemplo a seguir agruparemos pela coluna method e contaremos os valores válidos da coluna orbital_period
planets.groupby('method')['orbital_period'].count()

method
Astrometry                         2
Eclipse Timing Variations          9
Imaging                           12
Microlensing                       7
Orbital Brightness Modulation      3
Pulsar Timing                      5
Pulsation Timing Variations        1
Radial Velocity                  553
Transit                          397
Transit Timing Variations          3
Name: orbital_period, dtype: int64

In [4]:
# No exemplo abaixo agrupamos por método e ano e contamos os valores das demais colunas.
# Mostramos apenas as primeiras 10 linhas para a visualização não ficar muito grande
planets2 = planets.groupby(['method', 'year']).count()
planets2.head(10)

number  orbital_period  mass  distance
method                    year                                        
Astrometry                2010       1               1     0         1
                          2013       1               1     0         1
Eclipse Timing Variations 2008       2               2     0         2
                          2009       1               1     1         0
                          2010       2               2     0         2
                          2011       3               3     0         0
                          2012       1               1     1         0
Imaging                   2004       3               0     0         3
                          2005       1               0     0         1
                          2006       4               1     0         2

**Aggregate, filter, transform, apply**  
Em particular, os objetos GroupBy possuem os métodos ``aggregate()``, ``filter()``, ``transform()`` e ``apply()`` que implementam eficientemente uma variedade de operações úteis antes de combinar os dados agrupados.

**Aggregate**  
Agora, estamos familiarizados com as agregações do GroupBy com ``sum()``, ``median()`` e semelhantes, mas o método ``aggregate()`` permite ainda mais flexibilidade. Podemos usar uma string, uma função ou uma lista e calcular todos os agregados de uma só vez.

In [5]:
planets.groupby('method').aggregate({'distance': ['min', 'mean', 'max'], 'orbital_period': [np.median]})

distance                       orbital_period
                                   min         mean      max         median
method                                                                     
Astrometry                       14.98    17.875000    20.77     631.180000
Eclipse Timing Variations       130.72   315.360000   500.00    4343.500000
Imaging                           7.69    67.715937   165.00   27500.000000
Microlensing                   1760.00  4144.000000  7720.00    3300.000000
Orbital Brightness Modulation  1180.00  1180.000000  1180.00       0.342887
Pulsar Timing                  1200.00  1200.000000  1200.00      66.541900
Pulsation Timing Variations        NaN          NaN      NaN    1170.000000
Radial Velocity                   1.35    51.600208   354.00     360.200000
Transit                          38.00   599.298080  8500.00       5.714932
Transit Timing Variations       339.00  1104.333333  2119.00      57.011000

**Filter**  
Uma operação de filtragem permite descartar dados com base nas propriedades do grupo.

Nota: Para que o filter funcione é necessário que se crie uma função de validação. A função ``filter()`` deve retornar um valor booleano especificando se o grupo passa pela filtragem.

In [6]:
# No exemplo abaixo, traremos somente os valores agrupados pela coluna "method" cuja a contagem do valores válidos de
# "orbital_period" forem inferiores ou iguas à 3. Se tiver dúvida revise os valores trazidos no primeiro exemplo
# groupby executado anteriormente:
print('relembrando os valores do groupby:', chr(10), planets.groupby('method')['orbital_period'].count())

planets.groupby('method').filter(lambda x: x['orbital_period'].count() <= 3)

relembrando os valores do groupby: 
 method
Astrometry                         2
Eclipse Timing Variations          9
Imaging                           12
Microlensing                       7
Orbital Brightness Modulation      3
Pulsar Timing                      5
Pulsation Timing Variations        1
Radial Velocity                  553
Transit                          397
Transit Timing Variations          3
Name: orbital_period, dtype: int64


,method,number,orbital_period,mass,distance,year
113,Astrometry,1,246.360000,NaN,20.77,2013
537,Astrometry,1,1016.000000,NaN,14.98,2010
680,Transit Timing Variations,2,160.000000,NaN,2119.00,2011
736,Transit Timing Variations,2,57.011000,NaN,855.00,2012
749,Transit Timing Variations,3,NaN,NaN,NaN,2014
787,Orbital Brightness Modulation,2,0.240104,NaN,1180.00,2011
788,Orbital Brightness Modulation,2,0.342887,NaN,1180.00,2011
792,Orbital Brightness Modulation,1,1.544929,NaN,NaN,2013
813,Transit Timing Variations,2,22.339500,NaN,339.00,2013
958,Pulsation Timing Variations,1,1170.000000,NaN,NaN,2007


**Transform**  
Enquanto a agregação deve retornar uma versão reduzida dos dados, a transformação pode retornar uma versão transformada. Para essa transformação, a saída tem a mesma forma que a entrada. Um exemplo comum é centralizar os dados subtraindo a média do grupo.

In [7]:
# Criando um DataFrame
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'], 
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                  columns = ['key', 'data1', 'data2'])
print(df)

df.groupby('key').transform(lambda x: x - x.mean())

  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9


,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


**Apply**  
O método ``apply()`` permite aplicar uma função arbitrária aos resultados do grupo. A função deve pegar um DataFrame e retornar um objeto Pandas (por exemplo, DataFrame, Series); a operação de combinação será adaptada ao tipo de saída retornada.

In [8]:
def norm_by_data2(x):
    # x is a DataFrame of group values
    x['data1'] /= x['data2'].sum()
    return x

print(df.groupby('key').apply(norm_by_data2))

  key     data1  data2
0   A  0.000000      5
1   B  0.142857      0
2   C  0.166667      3
3   A  0.375000      3
4   B  0.571429      7
5   C  0.416667      9


**Exemplo combinado**  
Podemos executar combinações dos exemplos anteriores e montar uma tabela resumo.  
A seguir, vamos agrupar em linha (mehtod) e coluna (decade), e contar os planetas, fazendo um resumo.

In [9]:
# Preparando as counas de década
decade = 10 * (planets['year'] // 10) # Transformando o ano em número inteiro
decade = decade.astype(str) + 's' #Transformando em string
decade.name = 'decade'

# Agrupando os valores
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0
